<a href="https://colab.research.google.com/github/man-of-weirdness/MLnotebook/blob/main/notebook3bbee03806.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'vulnerable-c-source-code:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4785836%2F8103533%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240731%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240731T143147Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D65e63da850b8c4aef1d823071802dba0ae7f65d4d803e83b05b6267b501d40beae4c23aedaeda2ca2849ccd6fa7c4cd516ad3c23d44d22ebe737d552b8e9474ba12fd9a7e1d5be9a6f6ed193bd0e15635dab338ab6aa735644215e9495d0e14d4334e3f4944f2002886224f9c10545c476e7dfd369ee08c1e7eddea9a09f0a024f9466c36bd0622c0f4f994133f7c7c58f5add2e022734d8060dbf5cf92b392b20b539be930ee5c06b86ca7be6e4cbd7c531ea4d9d57ef1e137a7273b3b1edd6fbc15ccdd67986b53b803683252e42b2fa350a28909d12229590d0d96269a4d39c3934a5059817cc7e9aece8a7265ca5e806953edd23706f8f42c5b4768ab78a,gemma/keras/gemma_1.1_instruct_7b_en/3:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-models-data%2F22049%2F27858%2Fbundle%2Farchive.tar.gz%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240731%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240731T143147Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D50fc14f4c13e5813de1a77dff00fd6a009ea495ebe317f1e1a635e2fefd392eb9dfea8d76a0258542ed9e64d8ce9455f51c62234924ceeb42e16c3bee6290b08411515942e7f4fad7cda5a9637513143aa1735b104a2563a8d22901fe22e5356f656d33494f861067581b1d1644b57cbfd68d6a8e04b4b9c70ad876c574d64f5dcb45cb32fd62431289148d6d3fefb62752caf2d738ad3fb937f3e99e11247401eed47da90156b4f5eb8f8be597d809538ceb16800f0360c88ab863ea357f801729864f7f2ec3519d6f6c4f5a176b1a4691b3b133e88b7c13949c56cb0ac27228aa8c9dbe1a1e10b9c2436a700f4a0103bdf2feaf8aba584e66938b57febdeb9'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 25013129 bytes downloaded
Downloaded and uncompressed: vulnerable-c-source-code
[=======                                           ] 2122219520 bytes downloaded

In [ ]:
!pip install transformers datasets

import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import tokenizer_from_json
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import load_dataset, Dataset
from sklearn.model_selection import train_test_split

os.environ['CUDA_VISIBLE_DEVICES'] = '-1'  # Disable GPU
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Suppress TensorFlow warnings

# Function to load files from a directory
def load_files_from_directory(directory, label):
    file_contents = []
    for filename in os.listdir(directory):
        filepath = os.path.join(directory, filename)
        if os.path.isfile(filepath):
            with open(filepath, 'r', encoding='utf-8') as file:
                file_contents.append((file.read(), label))
    return file_contents

# Paths to the datasets
non_vulnerable_dir = '/kaggle/input/vulnerable-c-source-code/Dataset_raw/Non_vulnerable'
vulnerable_dir = '/kaggle/input/vulnerable-c-source-code/Dataset_raw/Vulnerable'



2024-07-31 11:27:27.856024: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-31 11:27:27.856222: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-31 11:27:28.001921: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
# Load data
vulnerable_files = load_files_from_directory(vulnerable_dir, 1)
non_vulnerable_files = load_files_from_directory(non_vulnerable_dir, 0)

data = vulnerable_files + non_vulnerable_files
np.random.shuffle(data)

# Create a DataFrame and split into train, validation, and test sets
df = pd.DataFrame(data, columns=['text', 'label'])
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)



In [ ]:
# Path to your model
model_path = '/kaggle/input/gemma/keras/gemma_1.1_instruct_7b_en/3'

# Verify if the model path contains necessary files
if not all(os.path.isfile(os.path.join(model_path, f)) for f in ['config.json', 'model.weights.h5', 'tokenizer.json']):
    raise FileNotFoundError("One or more necessary model files are missing. Ensure 'config.json', 'model.weights.h5', and 'tokenizer.json' are present in the model directory.")



In [ ]:
!pip install bitsandbytes==0.43.3

^C


In [ ]:
# Load the tokenizer and model from the local directory
import json
import keras_nlp
import bitsandbytes

bnb_config = BitsAndBytesConfig(
  load_in_4bit=True,
  bnb_4bit_quant_type="nf4",
  bnb_4bit_compute_dtype=torch.float16,
)

model = keras_nlp.models.GemmaCausalLM.from_preset(model_path)
model.backbone.enable_lora(rank=4)
model.summary()

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)

tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_val = val_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)

# Set up training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,  # Adjust based on memory limits
    per_device_eval_batch_size=2,
    num_train_epochs=1,  # Set to 1 for faster testing
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
)

# Train the model
trainer.train()

# Evaluate the model
results = trainer.evaluate(tokenized_test)
print(results)


ModuleNotFoundError: No module named 'bitsandbytes'